In [ ]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import skbio.io as skbio
import skbio.diversity.alpha as alpha_diversity
import statsmodels.api as sm
import seaborn as sns
import cpi
import math
#cpi.update()

In [ ]:
#df_2012 = pd.read_csv('survey_results2012.csv',encoding= 'unicode_escape')
#df_2013 = pd.read_csv('survey_results_public2013.csv',encoding= 'unicode_escape', low_memory=False)
#df_2014 = pd.read_csv('survey_results_public2014.csv',encoding= 'unicode_escape', low_memory=False)
#df_2015 = pd.read_csv('survey_results_public2015.csv',encoding= 'unicode_escape',low_memory=False)
df_2016 = pd.read_csv('survey_results_public2016.csv',encoding= 'unicode_escape')
df_2017 = pd.read_csv('survey_results_public2017.csv')
df_2018 = pd.read_csv('survey_results_public2018.csv', low_memory=False)
df_2019 = pd.read_csv('survey_results_public2019.csv')
df_2020 = pd.read_csv('survey_results_public2020.csv')
df_2021 = pd.read_csv('survey_results_public2021.csv')
df_2022 = pd.read_csv('survey_results_public2022.csv')
df_2016.rename(columns={'gender': 'Gender'}, inplace=True)
df_2016.rename(columns={'age': 'Age'}, inplace=True)

In [ ]:
pd.set_option('display.max_columns', None)
column_names = df_2016.columns.to_list()
column_names_str = ', '.join(column_names)
print(column_names_str)

In [ ]:
##Definir a função para calcular o coeficiente de gini
def gini(data):
    return alpha_diversity.gini_index(data, method='rectangles')

##Definir a função para criar o dicionario de mapeamento
def createMappingDict(data, column_name):
    counts = data[column_name].value_counts()
    replacement_values = range(1, len(counts)+1)
    # Create a dictionary to map the unique count strings to the replacement values
    mapping_dict = dict(zip(counts.index, replacement_values))
    return mapping_dict

##Função para preparar o dataset para o coeficiente de Gini:
def cleanDataset(data, value_to_drop, column_name):
    if column_name not in data.columns:
        raise ValueError(f"Column '{column_name}' does not exist in the DataFrame.") 
    replacement_dict = createMappingDict(data, column_name)
    data_clean = data.dropna(subset=[column_name]).copy()
    
    if (value_to_drop != ''):
        data_clean = data_clean[data_clean[column_name] != value_to_drop]
        
    data_clean[column_name] = data_clean[column_name].map(replacement_dict)
    return data_clean

def cleanAllDatasetsGivenColumn (datasets, column_name):
    datasets_clean = []
    for i, dataset in enumerate(datasets):
        dataset_clean = cleanDataset(dataset,'',column_name)
        datasets_clean.append(dataset_clean)
    return datasets_clean

def getGiniForAllDatasetsGivenColumn (datasets, column_name):
    giniResults = []
    datasets_clean = cleanAllDatasetsGivenColumn(datasets, column_name)
    for i, dataset in enumerate(datasets_clean):
        gini_c = gini(dataset[column_name])
        giniResults.append(gini_c)
    return giniResults
def getAgeRange(age):
    if age >= 65:
        return "65 or older"
    elif age >= 55:
        return "55-64 years"
    elif age >= 45:
        return "45-54 years"
    elif age >= 35:
        return "35-44 years"
    elif age >= 25:
        return "25-34 years"
    elif age >= 18:
        return "18-24 years"
    else:
        return "Under 18"

def applyAgeRange(df):
    df['Age Range'] = df['Age'].apply(get_age_range)
##Definir a função para calcular o coeficiente de gini
def gini(data):
    return alpha_diversity.gini_index(data, method='rectangles')

##Definir a função para criar o dicionario de mapeamento
def createMappingDict(data, column_name):
    counts = data[column_name].value_counts()
    replacement_values = range(1, len(counts)+1)
    # Create a dictionary to map the unique count strings to the replacement values
    mapping_dict = dict(zip(counts.index, replacement_values))
    return mapping_dict

def createMappingDictForSet (setList):
    replacement_values = range(1, len(setList)+1)
    # Create a dictionary to map the unique count strings to the replacement values
    mapping_dict = dict(zip(setList, replacement_values))
    return mapping_dict

def PlotAverage (df, xlab, ylab,fltrtitle, fltr):
    df = df.dropna(subset=[xlab])
    df = df.dropna(subset=[ylab])
    df_use = df
    if fltr != '':
        df_use = df_2022[df_2022[fltrtitle] == fltr]
    avg = df_use.groupby(xlab)[ylab].mean()
    plt.plot(avg.index, avg.values, marker='o')
    plt.xlabel(xlab)
    plt.ylabel('Average '+ylab)
    plt.title('Average '+ylab + ' per ' + xlab + ' in the USA')
    plt.xticks(avg.index, rotation=45, fontsize=8)
    plt.tight_layout()
    plt.show()
    
    
def correctCompensationInflation(value, year):
    return cpi.inflate(value, year)

def adjustGenders(df):
    df['Gender'] = df['Gender'].astype(str)
    df['Gender'] = df['Gender'].replace('nan', np.nan)
    multiple_values = ['Male', 'Transgender']
    multiple_values2 = ['Female', 'Transgender']

    for index, row in df.iterrows():
        genders = row['Gender']

        # Skip the row if the 'Gender' value is NaN or a float
        if pd.isna(genders) or isinstance(genders, float):
            continue

        gender_list = genders.split(';')

        if len(gender_list) == 1:
            if 'Transgender' in gender_list:
                df.at[index, 'Gender'] = 'Other'
            elif 'Or, in your own words:' in gender_list:
                df.at[index, 'Gender'] = 'Other'
        elif len(gender_list) == 2:
            if all(value in gender_list for value in multiple_values):
                df.at[index, 'Gender'] = 'Male'
            elif all(value in gender_list for value in multiple_values2):
                df.at[index, 'Gender'] = 'Female'
            else:
                df.at[index, 'Gender'] = 'Multigender'
        else:
            df.at[index, 'Gender'] = 'Multigender'

    df['Gender'] = df['Gender'].dropna()
    
            
def adjustAge(df):
     for index, row in df.iterrows():
        age = row['Age']
        if age >= 65:
            df.at[index, 'Age'] = '65 or older'
        elif age >= 55:
            df.at[index, 'Age'] = '55-64 years'
        elif age >= 45:
            df.at[index, 'Age'] = '45-54 years'
        elif age >= 35:
            df.at[index, 'Age'] = '35-44 years'
        elif age >= 25:
            df.at[index, 'Age'] = '25-34 years'
        elif age >= 18:
            df.at[index, 'Age'] = '18-24 years'
        else:
            df.at[index, 'Age'] = 'Under 18'
        
def adjustSalary(df,SalaryColumn,year):
    df['monthly_salary'] = df[SalaryColumn].apply(lambda salary: correctCompensationInflation(salary, year) / 12)
    

def plotGiniThroughTheYears (coefficients, start_y, end_y,title):
    years = range(start_y, end_y+1)


    fig, ax = plt.subplots()


    ax.scatter(years, coefficients)

    ax.set_xlabel('Years')
    ax.set_ylabel('Gini coefficients')
    ax.set_title(title)

    plt.show()

def plotCounts(df, group1, group2, title, order=None, legend_order=None, legend_title=None):
    counts = df.groupby([group1, group2]).size().unstack(fill_value=0)
    
    if order is not None:
        counts = counts.reindex(order)
    
    if legend_order is not None:
        counts = counts.reindex(columns=legend_order)
    
    counts.plot(kind='bar', stacked=True)

    plt.xlabel(group1)
    plt.ylabel('Count')
    plt.title(title)

    handles, labels = plt.gca().get_legend_handles_labels()

    if legend_order is not None:
        handles = [handles[labels.index(label)] for label in legend_order]
        labels = legend_order

    legend = plt.legend(handles, labels, title=legend_title, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.subplots_adjust(right=0.8)  # Adjust the right margin to make space for the legend

    plt.show()

def plotAvgPerYear(datasets,group1, group2,year_start): 
    df_average = []
    df_average = pd.DataFrame(columns=["Ano", "Média"])
    for df in datasets:
        df = df.dropna(subset=[group2])
        avg = df.groupby(group1)[group2].mean()
        filtered_column = avg.filter(like='25')
        df_average = pd.concat([df_average, pd.DataFrame({"Ano":year_start, "Média": [filtered_column.iloc[0]]})], ignore_index=True)
        year_start+=1
    plt.scatter(df_average['Ano'], df_average['Média'], marker='o')
    #plt.plot(df_average['Ano'], df_average['Média'], marker='o')
    
    plt.xlabel('Ano')
    plt.ylabel('Média')
    plt.title('Média salarial anual de jovens de 25 a 34 anos')
    x_ticks = range(int(df_average['Ano'].min()), int(df_average['Ano'].max()) + 1)
    plt.xticks(x_ticks)
    plt.show()
    
def plotAvgPerYear2(datasets, group2, year_start, country=None):
    df_average = pd.DataFrame(columns=["Ano", "Média"])

    for df in datasets:
        if country:
            df = df[df['Country'] == country]
        df = df.dropna(subset=[group2])
        avg = df[group2].mean()
        df_average = pd.concat([df_average, pd.DataFrame({"Ano": [year_start], "Média": [avg]})], ignore_index=True)
        year_start += 1
    
    plt.scatter(df_average['Ano'], df_average['Média'], marker='o')
    
    plt.xlabel('Ano')
    plt.ylabel('Média')
    if country is not None:
        plt.title('Média salarial anual do Brasil')
    else:
        plt.title('Média salarial anual')
    x_ticks = range(int(df_average['Ano'].min()), int(df_average['Ano'].max()) + 1)
    plt.xticks(x_ticks)
    plt.show()
    
    

def printPercentage(df, group1, group2, title, order=None, legend_order=None, legend_title=None):
    total_counts = df.groupby(group1).size()
    grouped_counts = df.groupby([group1, group2]).size()
    percentages = (grouped_counts / total_counts * 100).unstack(fill_value=0)

    if order is not None:
        percentages = percentages.reindex(order)

    if legend_order is not None:
        percentages = percentages.reindex(columns=legend_order)

    for gender in percentages.index:
        gender_percentages = percentages.loc[gender]
        sorted_percentages = gender_percentages.loc[legend_order]
        print(f"Gender: {gender}")
        for opinion, percentage in sorted_percentages.items():
            print(f"{opinion}: {percentage:.2f}%")
        print()

In [ ]:
##Adjust genders
adjustGenders(df_2022)
adjustGenders(df_2021)
adjustGenders(df_2020)
adjustGenders(df_2019)
adjustGenders(df_2018)
adjustGenders(df_2017)
adjustGenders(df_2016)

##Adjust ages
adjustAge(df_2019)
adjustAge(df_2020)


##Adjust to inflation
adjustSalary(df_2022,'ConvertedCompYearly',2022)
adjustSalary(df_2021,'ConvertedCompYearly',2021)
adjustSalary(df_2020,'ConvertedComp',2020)
adjustSalary(df_2019,'ConvertedComp',2019)
adjustSalary(df_2018,'ConvertedSalary',2018)


##Adding column to explict survey year
#df_2017['survey_year'] = 2017
#df_2018['survey_year'] = 2018
#df_2019['survey_year'] = 2019
#df_2020['survey_year'] = 2020
#df_2021['survey_year'] = 2021
#df_2022['survey_year'] = 2022

##Create dataset list
datasets = []
#datasets.append(df_2016)
#datasets.append(df_2017)
datasets.append(df_2018)
datasets.append(df_2019)
datasets.append(df_2020)
datasets.append(df_2021)
datasets.append(df_2022)

##
datasets_clean = []
datasets_clean.append(df_2018)
datasets_clean.append(df_2019)
datasets_clean.append(df_2020)
datasets_clean.append(df_2021)
datasets_clean.append(df_2022)

##Create dataset list
datasets_g = []
datasets_g.append(df_2016)
datasets_g.append(df_2017)
datasets_g.append(df_2018)
datasets_g.append(df_2019)
datasets_g.append(df_2020)
datasets_g.append(df_2021)
datasets_g.append(df_2022)

##
datasets_brazil = []

datasets_brazil.append(df_2016[df_2016['country'] == 'Brazil'])
datasets_brazil.append(df_2017[df_2017['Country'] == 'Brazil'])
datasets_brazil.append(df_2018[df_2018['Country'] == 'Brazil'])
datasets_brazil.append(df_2019[df_2019['Country'] == 'Brazil'])
datasets_brazil.append(df_2020[df_2020['Country'] == 'Brazil'])
datasets_brazil.append(df_2021[df_2021['Country'] == 'Brazil'])
datasets_brazil.append(df_2022[df_2022['Country'] == 'Brazil'])

In [ ]:
ginicoefficients = getGiniForAllDatasetsGivenColumn(datasets_g,'Gender')
plotGiniThroughTheYears(ginicoefficients,2016,2022,"Coeficiente de gini para diversidade de gênero anual")

In [ ]:
ginicoefficients = getGiniForAllDatasetsGivenColumn(datasets_brazil,'Gender')
plotGiniThroughTheYears(ginicoefficients,2016,2022,"Coeficiente de gini para diversidade de gênero anual no Brasil")

In [ ]:
custom_order = ['65 years or older', '55-64 years old', '45-54 years old', '35-44 years old', '25-34 years old','18-24 years old','Under 18 years old','Prefer not to say']
legend_order = ['Very unfavorable','Unfavorable','Indifferent','Unsure','Favorable','Very favorable']
plotCounts(df_2022,'Age','Blockchain',"Opinião por idade sobre blockchain em 2022",custom_order,legend_order,"Opinião")

In [ ]:
custom_order = ['65 or older', '55-64 years', '45-54 years', '35-44 years', '25-34 years','18-24 years','Under 18']
legend_order = ['An irresponsible use of resources', 'A passing fad','Useful for decentralized currency (i.e., Bitcoin)','Useful for immutable record keeping outside of currency','Useful across many domains and could change many aspects of our lives']
plotCounts(df_2019,'Age','BlockchainIs',"Opinião por idade sobre blockchain em 2019",custom_order,legend_order,"Opinião")

In [ ]:
plotAvgPerYear(datasets_clean,'Age','monthly_salary',2018)

In [ ]:
plotAvgPerYear2(datasets_clean, 'monthly_salary', 2018,None)

In [ ]:
plotAvgPerYear2(datasets_clean, 'monthly_salary', 2018,'Brazil')

In [ ]:
def checkEmployeed(df, col, df_f, ano):
    df2 = df.dropna(subset=[col])
    df3 = df[~df['Employment'].str.strip().eq('Not employed, but looking for work')]
    df3 = df3[~df3['Employment'].str.strip().eq('Not employed, and not looking for work')]
    df3 = df3[~df3['Employment'].str.strip().eq('Retired')] 
    
    total_count = len(df)
    employed_count = len(df3)
    salary_count = len(df2)
    #df_f.loc[len(df_f)] = [ano, total_count, employed_count, salary_count,age[0]]
    df_f.loc[len(df_f)] = [ano, len(df), len(df3), len(df2)]
df_f = []
df_empregados = []
df_f = pd.DataFrame(columns=["Ano", "Total", "Empregados", "Salários"])
checkEmployeed(df_2018, 'ConvertedSalary', df_f, 2018)
checkEmployeed(df_2019, 'ConvertedComp', df_f, 2019)
checkEmployeed(df_2020, 'ConvertedComp', df_f, 2020)
checkEmployeed(df_2021, 'ConvertedCompYearly', df_f, 2021)
checkEmployeed(df_2022, 'ConvertedCompYearly', df_f, 2022)

df_f['Empregados (%)'] = df_f['Empregados'] / df_f['Total'] * 100
df_f['Salários (%)'] = df_f['Salários'] / df_f['Total'] * 100

ax = df_f.plot(x='Ano', y=['Empregados (%)', 'Salários (%)'], kind='bar')
plt.xlabel('Ano')
plt.ylabel('Porcentagem')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))  # Legend at the right side
plt.show()


In [ ]:
def checkEmployeed(df, col, df_f, ano):
    df2 = df.dropna(subset=[col])
    df3 = df2[~df2['Employment'].str.strip().eq('Not employed, but looking for work')]
    df3 = df3[~df3['Employment'].str.strip().eq('Not employed, and not looking for work')]
    df3 = df3[~df3['Employment'].str.strip().eq('Retired')]
    df_
    

In [ ]:
df = df_2018
df2 = df.dropna(subset=['ConvertedSalary'])
df3 = df2[~df2['Employment'].str.strip().eq('Not employed, but looking for work')]
df3 = df3[~df3['Employment'].str.strip().eq('Not employed, and not looking for work')]
df3 = df3[~df3['Employment'].str.strip().eq('Retired')]
#top10 = df3['DevType'].head(10)
df3 = df3.dropna(subset=['Gender'])
#df3['DevType'].value_counts().head(10)

In [ ]:
def getTheAverageSalaryGenderDevAge(df, salaryCol, devCol):

    df_filtered = df[['Age', 'Gender', devCol, salaryCol]]
    #df_filtered.dropna(subset=['Gender', devCol, salaryCol], inplace=True)
    top10_devtypes = df_filtered['DevType'].value_counts().head(12).index


    grouped_df = df_filtered.groupby(['Age', 'Gender', devCol])[salaryCol].mean().reset_index()
    grouped_df.rename(columns={salaryCol: 'Average Salary'}, inplace=True)


    count_df = df_filtered.groupby(['Age', 'Gender', devCol]).size().reset_index(name='Count')

    grouped_df = pd.merge(grouped_df, count_df, on=['Age', 'Gender', devCol], how='left')


    for age in grouped_df['Age'].unique():
        print(f"Age Group: {age}")
        print("------------------------------------")

        age_group_df = grouped_df[grouped_df['Age'] == age]


        for gender in age_group_df['Gender'].unique():
            print(f"Gender: {gender}")
            print("----------------------")


            gender_group_df = age_group_df[age_group_df['Gender'] == gender]

            
            for devtype in top10_devtypes:
               
                devtype_group_df = gender_group_df[gender_group_df[devCol] == devtype]


                if devtype_group_df.empty:
                    avg_salary = 'NONE'
                    count = 'NONE'
                else:

                    count = devtype_group_df['Count'].values[0]


                    avg_salary = devtype_group_df['Average Salary'].values[0]


                print(f"{devtype}: Average Salary: {avg_salary}, Count: {count}")

            print()


getTheAverageSalaryGenderDevAge(df_2018,'monthly_salary','DevType')

In [ ]:
def getTheAverageSalaryGenderDevAge(df, salaryCol, devCol,man):

    df_filtered = df[['Age', 'Gender', devCol, salaryCol]].copy()
    country = 'Brazil'
    df_filtered = df[df['Country'] == country][['Age', 'Gender', devCol, salaryCol]]
    #df_filtered.dropna(subset=['Gender', devCol, salaryCol], inplace=True)

    top10_devtypes = df_filtered['DevType'].value_counts().head(10).index

  
    grouped_df = df_filtered.groupby(['Age', 'Gender', devCol])[salaryCol].mean().reset_index()
    grouped_df.rename(columns={salaryCol: 'Average Salary'}, inplace=True)

    count_df = df_filtered.groupby(['Age', 'Gender', devCol]).size().reset_index(name='Count')

  
    grouped_df = pd.merge(grouped_df, count_df, on=['Age', 'Gender', devCol], how='left')

  
    man_avg_salary = grouped_df[grouped_df['Gender'] == man].groupby(['DevType', 'Age'])['Average Salary'].mean().reset_index()
    man_avg_salary.rename(columns={'Average Salary': 'Man Average Salary'}, inplace=True)


    for age in grouped_df['Age'].unique():
        print(f"Age Group: {age}")
        print("------------------------------------")


        age_group_df = grouped_df[grouped_df['Age'] == age]


        for gender in age_group_df['Gender'].unique():
            print(f"Gender: {gender}")
            print("----------------------")


            gender_group_df = age_group_df[age_group_df['Gender'] == gender]

            
            for devtype in top10_devtypes:
                devtype_group_df = gender_group_df[gender_group_df[devCol] == devtype]


                if devtype_group_df.empty:
                    continue 
                    

                count = devtype_group_df['Count'].values[0]

                avg_salary = devtype_group_df['Average Salary'].values[0]

              
                man_avg_salary_dev_age = man_avg_salary.loc[(man_avg_salary['DevType'] == devtype) & (man_avg_salary['Age'] == age), 'Man Average Salary']
                if man_avg_salary_dev_age.empty:
                    percent_difference = np.nan
                else:
                    man_avg_salary_dev_age = man_avg_salary_dev_age.values[0]
                    percent_difference = ((avg_salary - man_avg_salary_dev_age) / man_avg_salary_dev_age) * 100

   
                salary_values = df_filtered[(df_filtered['Age'] == age) & (df_filtered['Gender'] == gender) & (df_filtered[devCol] == devtype)][salaryCol].values
                
                salary_values = list(filter(lambda x: not math.isnan(x), salary_values))
 
                std_dev = np.std(salary_values, ddof=0)

        
                print(f"{devtype}: Average Salary: {avg_salary}, Count: {count}")
                if percent_difference is not None:
                    print(f"       % Difference to average 'Man' Gender in {devtype} DevType and {age} age range: {percent_difference:.2f}%")
                if std_dev is not None:
                    print(f"       Standard Deviation: {std_dev}")
                print()


getTheAverageSalaryGenderDevAge(df_2022, 'monthly_salary', 'DevType','Man')

In [ ]:
def printPercentage2(df, group1, group2, title, order=None, legend_order=None, legend_title=None):
    total_counts = df.groupby(group1).size()
    answered_counts = df[df[group2].notnull()].groupby(group1).size()
    grouped_counts = df.groupby([group1, group2]).size()
    percentages = (grouped_counts / answered_counts * 100).unstack(fill_value=0)

    if order is not None:
        percentages = percentages.reindex(order)

    if legend_order is not None:
        percentages = percentages.reindex(columns=legend_order)

    for gender in percentages.index:
        gender_percentages = percentages.loc[gender]
        sorted_percentages = gender_percentages.loc[legend_order]
        print(f"Gender: {gender}")
        for opinion, percentage in sorted_percentages.items():
            print(f"{opinion}: {percentage:.2f}%")
        print()
legend_order = ['Very unfavorable', 'Unfavorable', 'Indifferent', 'Unsure', 'Favorable', 'Very favorable']
printPercentage2(df_2022, 'Gender', 'Blockchain', "Opinião por genero sobre blockchain em 2022", None, legend_order, "Opinião")

In [ ]:
def printPercentage2(df, group1, group2, title, order=None, legend_order=None, legend_title=None):
    total_counts = df.groupby(group1).size()
    answered_counts = df[df[group2].notnull()].groupby(group1).size()  
    grouped_counts = df.groupby([group1, group2]).size()
    percentages = (grouped_counts / answered_counts * 100).unstack(fill_value=0)

    if order is not None:
        percentages = percentages.reindex(order)

    if legend_order is not None:
        percentages = percentages.reindex(columns=legend_order)

    for gender in percentages.index:
        gender_percentages = percentages.loc[gender]
        sorted_percentages = gender_percentages.loc[legend_order]
        print(f"Gender: {gender}")
        for opinion, percentage in sorted_percentages.items():
            print(f"{opinion}: {percentage:.2f}%")
        print()
legend_order = ['An irresponsible use of resources', 'A passing fad','Useful for decentralized currency (i.e., Bitcoin)','Useful for immutable record keeping outside of currency','Useful across many domains and could change many aspects of our lives']
printPercentage2(df_2019, 'Gender', 'BlockchainIs', "Opinião por genero sobre blockchain em 2022", None, legend_order, "Opinião")

In [ ]:
legend_order = ['Very unfavorable', 'Unfavorable', 'Indifferent', 'Unsure', 'Favorable', 'Very favorable']
printPercentage(df_2022, 'Gender', 'Blockchain', "Opinião por genero sobre blockchain em 2022", None, legend_order, "Opinião")

In [ ]:
legend_order = ['An irresponsible use of resources', 'A passing fad','Useful for decentralized currency (i.e., Bitcoin)','Useful for immutable record keeping outside of currency','Useful across many domains and could change many aspects of our lives']
printPercentage(df_2019, 'DevType', 'BlockchainIs', "Opinião por genero sobre blockchain em 2022", None, legend_order, "Opinião")

In [ ]:
def printAverageSalary(df, group1, group2, title, order=None, legend_order=None, legend_title=None):
    grouped_df = df.groupby(group2)['monthly_salary'].mean()
    
    if order is not None:
        grouped_df = grouped_df.reindex(order)
    
    if legend_order is not None:
        grouped_df = grouped_df.reindex(index=legend_order)
    
    print(title)
    for opinion, avg_salary in grouped_df.items():
        print(f"Average salary of {opinion}: {avg_salary:.2f}")

legend_order = ['Very unfavorable', 'Unfavorable', 'Indifferent', 'Unsure', 'Favorable', 'Very favorable']
printAverageSalary(df_2022, None, 'Blockchain', "Average salary by opinion on blockchain in 2022", None, legend_order, "Opinion")


In [ ]:

def printAverageCodingYears(df, group1, group2, title, order=None, legend_order=None, legend_title=None):
    df['YearsCode'] = pd.to_numeric(df['YearsCode'], errors='coerce')

    grouped_df = df.groupby(group2)['YearsCode'].mean().round(2)

    if order is not None:
        grouped_df = grouped_df.reindex(order)

    print(title)
    print('-' * len(title))
    print(grouped_df)

    if group1:
        pivot_df = df.pivot_table(index=group1, columns=group2, values='YearsCode', aggfunc=np.mean).round(2)
        pivot_df = pivot_df.reindex(legend_order, axis=1)
        print('\nAverage coding years by', group1, 'and', group2)
        print(pivot_df)


legend_order = ['Very unfavorable', 'Unfavorable', 'Indifferent', 'Unsure', 'Favorable', 'Very favorable']
printAverageCodingYears(df_2022, None, 'Blockchain', "Average coding years by opinion on blockchain in 2022")
print("\n")
llegend_order = ['Very unfavorable', 'Unfavorable', 'Indifferent', 'Unsure', 'Favorable', 'Very favorable']
printAverageCodingYears(df_2019, None, 'BlockchainIs', "Average coding years by opinion on blockchain in 2022")

In [ ]:
def avgSalarySatisfaction(df, colName):
    groups = {}

    
    for satisfaction, salary in zip(df[colName], df['monthly_salary']):
        if pd.notnull(salary) and np.isfinite(salary):
            if satisfaction not in groups:
                groups[satisfaction] = {'count': 0, 'total_salary': 0}
            
            groups[satisfaction]['count'] += 1
            groups[satisfaction]['total_salary'] += salary


    for satisfaction, values in groups.items():
        if values['count'] > 0:
            average_salary = values['total_salary'] / values['count']
            print(f"Average salary for {satisfaction}: {average_salary}")

avgSalarySatisfaction(df_2020, 'JobSat')
print("\n")
avgSalarySatisfaction(df_2019, 'JobSat')
print("\n")
avgSalarySatisfaction(df_2018, 'JobSatisfaction')

In [ ]:
def avgYearsCodeJobSat(df, colName,colName2):
    job_sat_categories = df[colName].unique()
    
    for category in job_sat_categories:

        df[colName2] = pd.to_numeric(df[colName2], errors='coerce')
        
        avg_years_code = df[df[colName] == category][colName2].mean()
        print(f"Average YearsCodePro for {category}: {avg_years_code}")


avgYearsCodeJobSat(df_2020, 'JobSat','YearsCodePro')
print("\n")
avgYearsCodeJobSat(df_2019, 'JobSat','YearsCodePro')

In [ ]:
def avgSalarySatisfaction(df, colName, country_filter=None):
    groups = {} 

    if country_filter:
        df = df[df['Country'] == country_filter]

  
    for satisfaction, salary in zip(df[colName], df['monthly_salary']):
        if pd.notnull(salary) and np.isfinite(salary):
            if satisfaction not in groups:
                groups[satisfaction] = {'count': 0, 'total_salary': 0}

            groups[satisfaction]['count'] += 1
            groups[satisfaction]['total_salary'] += salary


    order = ['Very satisfied', 'Slightly satisfied', 'Neither satisfied nor dissatisfied', 'Slightly dissatisfied', 'Very dissatisfied']


    for satisfaction in order:
        if satisfaction in groups and groups[satisfaction]['count'] > 0:
            average_salary = groups[satisfaction]['total_salary'] / groups[satisfaction]['count']
            print(f"Average salary for {satisfaction}: {average_salary}")
print("2020")
avgSalarySatisfaction(df_2020, 'JobSat')
print("\n")
print("2019")
avgSalarySatisfaction(df_2019, 'JobSat')
print("\n")
print("2018")
avgSalarySatisfaction(df_2018, 'JobSatisfaction')


In [ ]:
def avgSalarySatisfaction(df, colName, country_filter=None, order=None):
    groups = {}  

    if country_filter:
        df = df[df['Country'] == country_filter]

    df = df.dropna(subset=[colName])


    for satisfaction, salary in zip(df[colName], df['monthly_salary']):
        if pd.notnull(salary) and np.isfinite(salary):  
            if satisfaction not in groups:
                groups[satisfaction] = {'count': 0, 'total_salary': 0}

            groups[satisfaction]['count'] += 1
            groups[satisfaction]['total_salary'] += salary

    if order:
        for satisfaction in order:
            if satisfaction in groups and groups[satisfaction]['count'] > 0:
                count = groups[satisfaction]['count']
                average_salary = groups[satisfaction]['total_salary'] / count
                print(f"{satisfaction} ({count} people): Average salary {average_salary}")
    else:
        for satisfaction, values in groups.items():
            if values['count'] > 0:
                count = values['count']
                average_salary = values['total_salary'] / count
                print(f"{satisfaction} ({count} people): Average salary {average_salary}")
order = ['Very satisfied', 'Slightly satisfied', 'Neither satisfied nor dissatisfied', 'Slightly dissatisfied', 'Very dissatisfied']
print("2020")
print("Mundo")
avgSalarySatisfaction(df_2020, 'JobSat',order=order)
print("Brasil")
avgSalarySatisfaction(df_2020, 'JobSat', 'Brazil',order=order)
print("\n")

print("2019")
print("Mundo")
avgSalarySatisfaction(df_2019, 'JobSat',order=order)
print("Brasil")
avgSalarySatisfaction(df_2019, 'JobSat', 'Brazil',order=order)
print("\n")

print("2018")
print("Mundo")
order_2018 = ['Extremely satisfied ', 'Moderately satisfied', 'Slightly satisfied', 'Neither satisfied nor dissatisfied', 'Slightly dissatisfied', 'Moderately dissatisfied', 'Extremely dissatisfied']
avgSalarySatisfaction(df_2018, 'JobSatisfaction', None, order=order_2018)
print("Brasil")
avgSalarySatisfaction(df_2018, 'JobSatisfaction', 'Brazil',order=order_2018)

In [ ]:
def remotework(df, colName):
    cleaned_df = df.dropna(subset=[colName])  
    total_count = cleaned_df.shape[0]  
    
 
    option_counts = cleaned_df[colName].value_counts()
    for option, count in option_counts.items():
        percentage = (count / total_count) * 100
        print(f"Percentage of people in {option}: {percentage}%")


remotework(df_2019, 'WorkRemote')
print("\n")
remotework(df_2022, 'RemoteWork')

In [ ]:
def modeJobSatByGender(df, colN):
    mode_satisfaction = df.groupby('Gender')[colN].agg(lambda x: x.mode().iloc[0])
    return mode_satisfaction


mode_satisfaction_by_gender = modeJobSatByGender(df_2018,'JobSatisfaction')
print(mode_satisfaction_by_gender)
print("\n")
mode_satisfaction_by_gender = modeJobSatByGender(df_2019,'JobSat')
print(mode_satisfaction_by_gender)
print("\n")
mode_satisfaction_by_gender = modeJobSatByGender(df_2020,'JobSat')
print(mode_satisfaction_by_gender)


In [ ]:
def modeJobSatByNeuro(df, colN):
    mode_satisfaction = df.groupby('Gender')[colN].agg(lambda x: x.mode().iloc[0])
    return mode_satisfaction


print("2018:")
print("MOST COMMON JOB: ")
print(df_2018['DevType'].value_counts().head(1))
print("\n")
mode_satisfaction_by_gender = modeJobSatByGender(df_2018,'DevType')
print(mode_satisfaction_by_gender)
print("\n")
print("2019:")
print("MOST COMMON JOB: ")
print(df_2019['DevType'].value_counts().head(1))
print("\n")
mode_satisfaction_by_gender = modeJobSatByGender(df_2019,'DevType')
print(mode_satisfaction_by_gender)
print("\n")
print("2020:")
print("MOST COMMON JOB: ")
print(df_2020['DevType'].value_counts().head(1))
print("\n")
mode_satisfaction_by_gender = modeJobSatByGender(df_2020,'DevType')
print(mode_satisfaction_by_gender)
print("\n")
print("2021:")
print("MOST COMMON JOB: ")
print(df_2021['DevType'].value_counts().head(1))
print("\n")
mode_satisfaction_by_gender = modeJobSatByGender(df_2021,'DevType')
print(mode_satisfaction_by_gender)
print("\n")
print("2022:")
print("MOST COMMON JOB: ")
print(df_2022['DevType'].value_counts().head(1))
print("\n")
mode_satisfaction_by_gender = modeJobSatByGender(df_2022,'DevType')
print(mode_satisfaction_by_gender)

In [ ]:
def calculate_avg_job_satisfaction(df, n,colName):
    df_filtered = df.dropna(subset=['DevType', colName])
    top_dev_types = df_filtered['DevType'].value_counts().head(n).index
    df_filtered = df_filtered[df_filtered['DevType'].isin(top_dev_types)]
    df_filtered[colName] = df_filtered[colName].map({'Very dissatisfied': 1, 'Slightly dissatisfied': 2, 'Neither satisfied nor dissatisfied': 3, 'Slightly satisfied': 4, 'Very satisfied': 5})
    avg_job_satisfaction = df_filtered.groupby('DevType')[colName].mean()
    return avg_job_satisfaction
avg_job_satisfaction = calculate_avg_job_satisfaction(df_2018, 10,'JobSatisfaction')
print(avg_job_satisfaction)
print("\n")
avg_job_satisfaction = calculate_avg_job_satisfaction(df_2019, 10,'JobSat')
print(avg_job_satisfaction)
print("\n")
avg_job_satisfaction = calculate_avg_job_satisfaction(df_2020, 10,'JobSat')
print(avg_job_satisfaction)


In [ ]:
def get_most_common_devtype(df):
    
    df_cleaned = df.dropna(subset=['MentalHealth', 'DevType'])

    
    top10_mentalhealth = df_cleaned['MentalHealth'].value_counts().head(10).index.tolist()

    
    df_filtered = df_cleaned[df_cleaned['MentalHealth'].isin(top10_mentalhealth)]

    
    most_common_devtype = {}

    
    for mental_health_category in top10_mentalhealth:
        
        df_category = df_filtered[df_filtered['MentalHealth'] == mental_health_category]

        most_common_devtype_category = df_category['DevType'].value_counts().head(3).index.tolist()

        most_common_devtype[mental_health_category] = most_common_devtype_category

    
    result_string = ''
    for category, devtypes in most_common_devtype.items():
        result_string += f"{category}:\n"
        for devtype in devtypes:
            result_string += f"{devtype}\n"
        result_string += '\n'

    return result_string

most_common_devtype = get_most_common_devtype(df_2022)
print(most_common_devtype)
print("\n\n")
most_common_devtype = get_most_common_devtype(df_2021)
print(most_common_devtype)

In [ ]:
def average_salary_by_remote_work(df, col_name, salary_col):
    cleaned_df = df.dropna(subset=[col_name, salary_col])

    average_salary = cleaned_df.groupby(col_name)[salary_col].mean()

    return average_salary

average_salary_work_remote = average_salary_by_remote_work(df_2019, 'WorkRemote', 'monthly_salary')
print("Average salary by WorkRemote:")
print(average_salary_work_remote)
print("\n")

average_salary_remote_work = average_salary_by_remote_work(df_2022, 'RemoteWork', 'monthly_salary')
print("Average salary by RemoteWork:")
print(average_salary_remote_work)


In [ ]:
def average_salary_by_remote_work_and_age(df, col_name, salary_col, age_col):
    cleaned_df = df.dropna(subset=[col_name, salary_col, age_col]) 
    
    average_salary = cleaned_df.groupby([col_name, age_col])[salary_col].mean()

    return average_salary
average_salary_work_remote_age = average_salary_by_remote_work_and_age(df_2019, 'WorkRemote', 'monthly_salary', 'Age')
print("Average salary by WorkRemote and Age:")
print(average_salary_work_remote_age)
print("\n")

average_salary_remote_work_age = average_salary_by_remote_work_and_age(df_2022, 'RemoteWork', 'monthly_salary', 'Age')
print("Average salary by RemoteWork and Age:")
print(average_salary_remote_work_age)


In [ ]:
def average_salary_by_remote_work_and_age_brazil(df, col_name, salary_col, age_col):
    cleaned_df = df[df['Country'] == 'Brazil'].dropna(subset=[col_name, salary_col, age_col])
    
    average_salary = cleaned_df.groupby([col_name, age_col])[salary_col].mean()

    return average_salary
average_salary_work_remote_age_brazil = average_salary_by_remote_work_and_age_brazil(df_2019, 'WorkRemote', 'monthly_salary', 'Age')
print("Average salary by WorkRemote and Age (Brazil only):")
print(average_salary_work_remote_age_brazil)
print("\n")

average_salary_remote_work_age_brazil = average_salary_by_remote_work_and_age_brazil(df_2022, 'RemoteWork', 'monthly_salary', 'Age')
print("Average salary by RemoteWork and Age (Brazil only):")
print(average_salary_remote_work_age_brazil)


In [ ]:
def compare_average_salary(df, country_filter):
    
    general_df = df[df['Country'] != country_filter]

    
    average_salary_general = general_df['monthly_salary'].mean()
    count_general = general_df.shape[0]

    
    filtered_df = df[df['Country'] == country_filter]

    
    average_salary_filtered = filtered_df['monthly_salary'].mean()
    count_filtered = filtered_df.shape[0]

    
    percentage_difference = ((average_salary_filtered - average_salary_general) / average_salary_general) * 100

    
    print(f"Average salary (general excluding {country_filter}): {average_salary_general}")
    print(f"Number of people (general excluding {country_filter}): {count_general}")
    print(f"Average salary ({country_filter}): {average_salary_filtered}")
    print(f"Number of people ({country_filter}): {count_filtered}")
    print(f"Percentage difference: {percentage_difference}%")

print("2O22")
compare_average_salary(df_2022, 'Brazil')
print("\n")
print("2O21")
compare_average_salary(df_2021, 'Brazil')
print("\n")
print("2O20")
compare_average_salary(df_2020, 'Brazil')
print("\n")
print("2O19")
compare_average_salary(df_2019, 'Brazil')
print("\n")
print("2O18")
compare_average_salary(df_2018, 'Brazil')
print("\n")

In [ ]:
def avgSalAccess(df, accessibility_col, salary_col, country_filter=None):
    
    if country_filter is not None:
        filtered_df = df[df['Country'] == country_filter]
    else:
        filtered_df = df.copy()

    
    filtered_df = filtered_df.dropna(subset=[accessibility_col, salary_col])

    for index, row in filtered_df.iterrows():
        access = row[accessibility_col]

    
        if pd.isna(access) or isinstance(access, float):
            continue

        access_list = access.split(';')

        if len(access_list) > 1:
            filtered_df.at[index, accessibility_col] = 'Multiple'

    
    accessibility_counts = filtered_df[accessibility_col].value_counts().to_dict()

    
    for group, count in accessibility_counts.items():
        group_df = filtered_df[filtered_df[accessibility_col] == group]
        avg_salary = group_df[salary_col].astype(float).mean()
        print("{} ({} person{}): Average salary {}".format(group, count, 's' if count > 1 else '', avg_salary))


print("2021")
print("Mundo")
avgSalAccess(df_2021,'Accessibility', 'monthly_salary')
print("\n")
print("Brasil")
avgSalAccess(df_2021,'Accessibility', 'monthly_salary','Brazil')
print("\n")
print("2022")
print("Mundo")
avgSalAccess(df_2022,'Accessibility', 'monthly_salary')
print("\n")
print("Brasil")
avgSalAccess(df_2022,'Accessibility', 'monthly_salary','Brazil')

In [ ]:
def accessCol(df, accessibility_col):
    filtered_df = df[df['Country'] == 'Brazil'].copy()
    filtered_df.loc[:, accessibility_col] = filtered_df[accessibility_col].astype(str)
    filtered_df.loc[:, accessibility_col] = filtered_df[accessibility_col].replace('nan', np.nan)
    
   
    filtered_df = filtered_df.dropna(subset=[accessibility_col])
    
    for index, row in filtered_df.iterrows():
        access = row[accessibility_col]
        
        if isinstance(access, str):
            access_list = access.split(';')
            if len(access_list) > 1:
                filtered_df.at[index, 'Access'] = 'Multiple'
            else:
                filtered_df.at[index, 'Access'] = access_list
        else:
            filtered_df.at[index, 'Access'] = np.nan

    print(filtered_df['Access'].value_counts())
print("2021")
#accessCol(df_2021, 'Accessibility')
df_2021['Access'].value_counts()

In [ ]:
print("2022")
#accessCol(df_2022, 'Accessibility')
df_2022['Access'].value_counts()

In [ ]:
accessCol(df_2022,'Accessibility')
accessCol(df_2021,'Accessibility')

In [ ]:
def mhCol(df, mh_col):
    df[mh_col] = df[mh_col].astype(str)
    df[mh_col] = df[mh_col].replace('nan', np.nan)
    df = df.dropna(subset=[mh_col])
    for index, row in df.iterrows():
        access = row[mh_col]
        
        if isinstance(access, str):
            access_list = access.split(';')
            if len(access_list) > 1:
                df.at[index, mh_col] = 'Multiple' 
            else:
                df.at[index, mh_col] = access_list
        else:
            df.at[index, mh_col] = np.nan  
    print(df[mh_col].value_counts()) 

print("2021")
mhCol(df_2021, 'MentalHealth')
print("\n")
print("2022")
mhCol(df_2022, 'MentalHealth')


In [ ]:
def mhCol2(df, mh_col):
    filtered_df = df[df['Country'] == 'Brazil'].copy()
    filtered_df.loc[:, mh_col] = filtered_df[mh_col].astype(str)
    filtered_df.loc[:, mh_col] = filtered_df[mh_col].replace('nan', np.nan)
    
    filtered_df = filtered_df.dropna(subset=[mh_col])
    
    for index, row in filtered_df.iterrows():
        access = row[mh_col]
        
        if isinstance(access, str):
            access_list = access.split(';')
            if len(access_list) > 1:
                filtered_df.at[index, 'Mh'] = 'Multiple'
            else:
                filtered_df.at[index, 'Mh'] = access_list
        else:
            filtered_df.at[index, 'Mh'] = np.nan

    print(filtered_df['Mh'].value_counts())
print("2022")
mhCol2(df_2022,'MentalHealth')
print("\n")
print("2021")
mhCol2(df_2021,'MentalHealth')

In [ ]:
print("2021")
print("Mundo")
avgSalAccess(df_2021,'MentalHealth', 'monthly_salary')
print("\n")
print("Brasil")
avgSalAccess(df_2021,'MentalHealth', 'monthly_salary','Brazil')
print("\n")
print("2022")
print("Mundo")
avgSalAccess(df_2022,'MentalHealth', 'monthly_salary')
print("\n")
print("Brasil")
avgSalAccess(df_2022,'MentalHealth', 'monthly_salary','Brazil')

In [ ]:
print("2021")
print("Mundo")
avgSalAccess(df_2021,'Accessibility', 'monthly_salary')
print("\n")
print("Brasil")
avgSalAccess(df_2021,'Accessibility', 'monthly_salary','Brazil')
print("\n")
print("2022")
print("Mundo")
avgSalAccess(df_2022,'Accessibility', 'monthly_salary')
print("\n")
print("Brasil")
avgSalAccess(df_2022,'Accessibility', 'monthly_salary','Brazil')

In [ ]:
def mostCommonLang(df, accessibility_col, language_col, country_filter=None, n=5):
    if country_filter:
        df_copy = df[df['Country'] == country_filter].copy()
    else:
        df_copy = df.copy()

    
    df_copy[language_col] = df_copy[language_col].str.replace('TypeScript', 'JavaScript').str.replace('Node.js', 'JavaScript', regex=False)

    
    df_copy['GroupedAccessibility'] = df_copy[accessibility_col].apply(lambda x: 'Multiple' if ';' in str(x) else x)

    
    df_copy = df_copy.dropna(subset=[accessibility_col, language_col])

    
    accessibility_counts = df_copy['GroupedAccessibility'].value_counts().sort_values(ascending=False)

    
    for group in accessibility_counts.index:
        group_df = df_copy[df_copy['GroupedAccessibility'] == group]

        if group == 'Multiple':
            print("Multiple ({} person{}):".format(len(group_df), 's' if len(group_df) > 1 else ''))
        else:
            print("{} ({} person{}):".format(group, len(group_df), 's' if len(group_df) > 1 else ''))

        languages = group_df[language_col].str.split(';').explode()

    
        if 'HTML/CSS' in languages.values:
            languages = languages[languages != 'HTML/CSS']
        if 'SQL' in languages.values:
            languages = languages[languages != 'SQL']

    
        language_counts = languages.value_counts().sort_values(ascending=False)

    
        top_languages = language_counts.head(n)

    
        print(top_languages)
        print()


bold_text = "\033[1m" + '2021' + "\033[0m"
print(bold_text)
bold_text = "\033[1m" + 'Mundo' + "\033[0m"
print(bold_text)
mostCommonLang(df_2021,'Accessibility', 'LanguageHaveWorkedWith')
bold_text = "\033[1m" + 'Brasil' + "\033[0m"
print(bold_text)
mostCommonLang(df_2021,'Accessibility', 'LanguageHaveWorkedWith','Brazil')
print("\n\n")
bold_text = "\033[1m" + '2022' + "\033[0m"
print(bold_text)
bold_text = "\033[1m" + 'Mundo' + "\033[0m"
print(bold_text)
mostCommonLang(df_2022,'Accessibility', 'LanguageHaveWorkedWith')
bold_text = "\033[1m" + 'Brasil' + "\033[0m"
print(bold_text)
mostCommonLang(df_2022,'Accessibility', 'LanguageHaveWorkedWith','Brazil')

In [ ]:
bold_text = "\033[1m" + '2021' + "\033[0m"
print(bold_text)
bold_text = "\033[1m" + 'Mundo' + "\033[0m"
print(bold_text)
mostCommonLang(df_2021,'MentalHealth', 'LanguageHaveWorkedWith')
bold_text = "\033[1m" + 'Brasil' + "\033[0m"
print(bold_text)
mostCommonLang(df_2021,'MentalHealth', 'LanguageHaveWorkedWith','Brazil')
print("\n\n")
bold_text = "\033[1m" + '2022' + "\033[0m"
print(bold_text)
bold_text = "\033[1m" + 'Mundo' + "\033[0m"
print(bold_text)
mostCommonLang(df_2022,'MentalHealth', 'LanguageHaveWorkedWith')
bold_text = "\033[1m" + 'Brasil' + "\033[0m"
print(bold_text)
mostCommonLang(df_2022,'MentalHealth', 'LanguageHaveWorkedWith','Brazil')

In [ ]:
def calculate_average_salary(dataframe, ed_col, salary_col, country=None):
    if country:
        filtered_df = dataframe[dataframe['Country'] == country]
    else:
        filtered_df = dataframe

    average_salary = filtered_df.groupby(ed_col)[salary_col].mean()
    average_salary = average_salary.sort_values(ascending=False)  
    return average_salary
print("2022")
avg = calculate_average_salary(df_2022,'EdLevel','monthly_salary','Brazil')
print(avg)
print("\n")
print("2021")
avg = calculate_average_salary(df_2021,'EdLevel','monthly_salary','Brazil')
print(avg)
print("\n")
print("2020")
avg = calculate_average_salary(df_2020,'EdLevel','monthly_salary','Brazil')
print(avg)
print("\n")
print("2019")
avg = calculate_average_salary(df_2019,'EdLevel','monthly_salary','Brazil')
print(avg)
print("\n")
print("2018")
avg = calculate_average_salary(df_2018,'FormalEducation','monthly_salary','Brazil')
print(avg)
print("\n")

In [ ]:
print("2022")
avg = calculate_average_salary(df_2022,'EdLevel','monthly_salary')
print(avg)
print("\n")
print("2021")
avg = calculate_average_salary(df_2021,'EdLevel','monthly_salary')
print(avg)
print("\n")
print("2020")
avg = calculate_average_salary(df_2020,'EdLevel','monthly_salary')
print(avg)
print("\n")
print("2019")
avg = calculate_average_salary(df_2019,'EdLevel','monthly_salary')
print(avg)
print("\n")
print("2018")
avg = calculate_average_salary(df_2018,'FormalEducation','monthly_salary')
print(avg)
print("\n")